In [124]:
import requests 
import tweepy
import json
import datetime
import pandas as pd
import sqlite3

In [87]:
with open('credentials.json', 'r') as fp:
    api_cred = json.load(fp)
    
CONSUMER_KEY = api_cred["CONSUMER_KEY"]
CONSUMER_SECRET = api_cred["CONSUMER_SECRET"]
OAUTH_TOKEN = api_cred["OAUTH_TOKEN"]
OAUTH_TOKEN_SECRET = api_cred["OAUTH_SECRET"]
BEARER_TOKEN = api_cred["BEARER_TOKEN"]

In [88]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [110]:
now = f"{datetime.datetime.today().strftime('%Y-%m-%d')}T00:00:00Z"
print(now)

yesterday = f"{((datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y-%m-%d'))}T00:00:00Z"
print(yesterday)

2022-09-18T00:00:00Z
2022-09-17T00:00:00Z


In [126]:
# Everything in the last 24 hours
# amount of tweets DONE 
# retweets received DONE
# likes received DONE
# comments received DONE
# followers gained DONE
# engagement rate CANNOT BE DONE (NEED ENGAGEMENT API WHICH IS NOT FREE)

def get_user_data(user_name='Renate_KE'):
    user_data = {}
    user = client.get_user(username=user_name, user_fields='public_metrics')
    user_id = user.data.id

    today = f"{datetime.datetime.today().strftime('%Y-%m-%d')}T00:00:00Z"
    yesterday = f"{((datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y-%m-%d'))}T00:00:00Z"

    tweets = client.get_users_tweets(id=user_id, exclude="retweets", start_time=yesterday, end_time=today, 
                                     tweet_fields=['public_metrics','author_id','conversation_id'], 
                                     max_results=100)

    followers = user.data['public_metrics']['followers_count']
    retweets = 0
    likes = 0
    replies = 0
    for t in tweets.data:
        retweets += t.data["public_metrics"]["retweet_count"]
        likes += t.data["public_metrics"]["like_count"]
        replies += t.data["public_metrics"]["reply_count"]
        # print(t.data["public_metrics"])

    user_data["tweets"] = len(tweets.data)
    user_data["retweets"] = retweets
    user_data["likes"] = likes
    user_data["replies"] = replies
    user_data["followers"] = followers
    user_data["time"] = today
    user_data["name"] = user_name
    
    return user_data

In [128]:
# insert data in a Dataframe 
# Later maybe create an SQLite DB and insert data
user_data_all = []
user_names = ['bordoni_russia', 'Linn_FPL', 'pap1pap', 'amrith']
for user in user_names:
    user_data_all.append(get_user_data(user))

df = pd.DataFrame(user_data_all)

In [129]:
df

,tweets,retweets,likes,replies,followers,time,name
0,16,205,525,195,12334,2022-09-18T00:00:00Z,bordoni_russia
1,15,5,601,102,24316,2022-09-18T00:00:00Z,Linn_FPL
2,50,15,763,107,10268,2022-09-18T00:00:00Z,pap1pap
3,6,2,55,8,10434,2022-09-18T00:00:00Z,amrith
